# Hugging Face PEFT を利用したアダプタの学習

Parameter-Efficient Fine Tuning (PEFT)メソッドは，事前学習済みモデルのパラメタをファインチューニング中にフリーズし，その上に学習可能なパラメータ（アダプタ）を追加する方法です．
アダプタは，タスク固有の情報を効率よく学習することが期待されます．
この方法は，メモリ使用量が少なく，完全にファインチューニングされたモデルと比較して計算リソースを低く抑えつつ，同等の結果が得られることが示されています．

PEFTで学習されたアダプタは，完全なモデルのサイズよりも1桁小さく，共有，保存，読み込むのが便利です．

> Hubに格納されている OPTForCausalLM モデルのアダプタの重みは約6MBで，モデルの全体の重みのサイズ約700MBに対して1%程度のサイズです．

Hugging Face PEFTについて詳しく知りたい場合は[ドキュメント](https://huggingface.co/docs/peft/index)を参照してください．

## インストール

Hugging Face PEFT をインストールして始めましょう:
```
pip install peft
```

## サポート対象のPEFTモデル

Hugging Face Transformers は，いくつかのPEFT（Parameter Efficient Fine-Tuning）メソッドをネイティブにサポートしており，ローカルまたはHubに格納されたアダプタウェイトを簡単に読み込んで実行または学習することができます．
以下の手法がサポートされています．

- Low Rank Adapters (LoRA)
- IA3
- AdaLoRA

他の方法を使用したい場合，プロンプト学習やプロンプト調整などについて詳しく知りたい場合，または hugging Face PEFT ライブラリ全般については，[ドキュメント](https://huggingface.co/docs/peft/index)を参照してください．

## PEFTアダプタの読み込み

Hugging Face TransformersからPEFTアダプタモデルを読み込んで使用するには，Hubリポジトリまたはローカルディレクトリにadapter_config.jsonファイルとアダプタウェイトが含まれていることを確認してください．
次に，`AutoModelFor?`クラスを使用してPEFTアダプタモデルを読み込むことができます．
例えば，因果的言語モデル用のアダプタを読み込むには:

1. PEFTモデルのIDを指定します
2. それを`AutoModelForCausalLM`クラスに渡します．

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "ybelkada/opt-350m-lora"
mdoel = AutoModelForCausalLM.from_pretrained(peft_model_id)

/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


> PEFTアダプタを`AutoModelFor?`クラスまたは基本モデルクラス（`OptForCausalLM`または`LlamaForCausalLM`など）で読み込むことができます．

また，`load_adapter`メソッドを呼び出すことで，PEFTアダプタを読み込むこともできます:

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "facebook/opt-350m"
peft_model_id = "ybelkada/opt-350m-lora"

model = AutoModelForCausalLM.from_pretrained(model_id)
mdoel.load_adapter(peft_model_id, adapter_name="adapter_1")

## 8bitまたは4bitでのロード

`bitsandbytes`との統合により，PEFTアダプタを8ビットまたは4ビットの精度で読み込むことができます．
これにより，大規模なモデルを読み込む際にメモリを節約することができます（[詳細](https://huggingface.co/docs/transformers/ja/quantization#bitsandbytes-integration)）
`from_pretrained()`の`quontization_config`パラメータで設定を変更し，`device="auto"`を指定することで，自動的に最適なデバイスにアダプタを配置することができます．


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "ybelkada/opt-350m-lora"
mdoel = AutoModelForCausalLM.from_pretrained(peft_model_id,
                                             config=BitsAndBytesConfig(load_in_8bit=True), # Load the model in 8-bit
                                             device_map="auto")

## 新しいアダプタを追加する

既存のアダプタを持つモデルに新しいアダプタを追加するために，`add_adapter`メソッドを使用することができます．
ただし，新しいアダプタは既存のアダプタと同じタイプである必要があります．
例えば，既存のモデルにLoRAアダプタがアタッチされている場合:

In [11]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False,
)

model.add_adapter(lora_config, adapter_name="adapter_1")

新しいアダプタを追加するには:

In [12]:
# attach new adapter with same config
model.add_adapter(lora_config, adapter_name="adapter_2")

`set_adapter`メソッドを使用して，どのアダプタを使用するかを指定します:

In [ ]:
# use_adapter_1
model.set_adapter("adapter_1")
output = model.generate(**inputs)
